## **Importação das bibliotecas e dados (AEP e Duas Unas)**

In [1]:
# %pip install ipykernel --break-system-packages
# %pip install scikit-learn --break-system-packages
# %pip install matplotlib --break-system-packages
# %pip install keras --break-system-packages
# %pip install tensorflow --break-system-packages
# %pip install pandas --break-system-packages
# %pip install plotly --break-system-packages
# %pip install seaborn --break-system-packages
# %pip install nbformat --break-system-packages
# %pip install keras --break-system-packages
# %pip install pswarm


In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.initializers import GlorotUniform    #Inicializador de pesos
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from scipy.signal import find_peaks

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf #Biblioteca para machine learning

In [ ]:

base_path = './dataset/'

# carregando arquivo CSV
time_base = 'days'  # Escolha do tipo de agrupamento
duas_unas_dataset = f'{base_path}duas_unas/{time_base}/grouped_1_{time_base}.csv'  # Definição do caminho do diretório do dataset
print(len(duas_unas_dataset))
validation_interval = 61  # Definição da parcela dos dados que serão retiradas do treinamento/teste para validação
time_step = 1  # Number of previous time steps to consider
output_steps = 60  # Number of future steps to predict

# transformando CSV em DataFrame
duas_unas_df = pd.read_csv(duas_unas_dataset)  # Transformação em dataset

duas_unas_validation = duas_unas_df.tail(validation_interval)  # Criação do dataset de validação

indices_to_validate = duas_unas_df.index[-validation_interval:].tolist()  # Obtenção dos índices dos dadoss de validação
duas_unas_df = duas_unas_df.drop(indices_to_validate).reset_index(drop=True)  # Retirada dos dados de validação do dataset original de treino/teste

duas_unas_df['timestamp'] = pd.to_datetime(duas_unas_df['timestamp'])  # Transformação dos dados da coluna 'timestamp' no formato datetime. Ela lida com vários formatos de data/hora e tenta interpretá-los de forma inteligente.
duas_unas_validation['timestamp'] = pd.to_datetime(duas_unas_validation['timestamp'])  # Idem da ação da linha de cima.

duas_unas_df['index'] = range(len(duas_unas_df))  # Criação da coluna 'index' em que os valores vão de zero ao tamanho total do daframe
duas_unas_validation['index'] = indices_to_validate  # Idem da acima da linha acima
print(indices_to_validate)
duas_unas_df['kwh'] = duas_unas_df['kWh fornecido']  # Criação da coluna 'kwh' que copia os dados da coluna 'kWh fornecido'
duas_unas_validation['kwh'] = duas_unas_validation['kWh fornecido']  # Idem da acima da linha acima

duas_unas_df.drop(columns=['timestamp'], inplace=True)  # Remoção da coluna 'timestamp', inclusive no dataframe original
duas_unas_validation.drop(columns=['timestamp'], inplace=True)  # Idem da acima da linha acima

duas_unas_df.drop(columns=['kWh fornecido'], inplace=True)  # Remoção da coluna 'kWh fornecido', inclusive no dataframe original
duas_unas_validation.drop(columns=['kWh fornecido'], inplace=True)  # Idem da acima da linha acima

X = duas_unas_df['index'].values.reshape(-1,1)  # Cria o array X com os índices do dataframe duas_unas_df
y = duas_unas_df['kwh'].values  # Cria o array y com os dados de energia de duas_unas_df
#print(y)
X_validation = duas_unas_validation['index'].values.reshape(-1,1)  # Cria o array X_validation com os índices do dataframe duas_unas_validation
y_validation = duas_unas_validation['kwh'].values  # Cria o array y com os dados de energia de duas_unas_validation
#print(X_validation)

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# regr = MLPRegressor(hidden_layer_sizes=(50,50,50), max_iter=200).fit(X_train, y_train)

# y_pred = regr.predict(X_test)

# from sklearn.metrics import mean_squared_error

# rms = mean_squared_error(y_test, y_pred, squared=False)

# plt.figure(figsize=(8, 6))
# plt.scatter(X_test, y_test, color="blue", label="Valores Reais")
# plt.scatter(X_test, y_pred, color="red", label="Previsões", alpha=0.6)
# plt.title(f"Teste do MLPRegressor (RMSE: {rms:.2f})")
# plt.xlabel("Horas (X)")
# plt.ylabel("Previsão (y)")
# plt.legend()
# plt.show()

In [5]:
# print(X_validation)
# y_pred_validation = regr.predict(X_validation)

# plt.figure(figsize=(8, 6))
# plt.scatter(X_validation, y_validation, color="blue", label="Valores Reais")
# plt.scatter(X_validation, y_pred_validation, color="red", label="Previsões", alpha=0.6)
# plt.title(f"Hora da Verdade")
# plt.xlabel("Hora (X)")
# plt.ylabel("Consumo (y)")
# plt.legend()
# plt.show()

In [6]:
# pirapama_dataset = f'{base_path}outras_unidades/pirapama/agrupado_hora_Pirapama_2021.csv'
# # transformando CSV em DataFrame
# pirapama_df = pd.read_csv(pirapama_dataset)

# pirapama_df['Data'] = pd.to_datetime(pirapama_df['Data'])

# pirapama_df['index'] = range(len(pirapama_df))

# pirapama_df['kwh'] = pirapama_df['kWh fornecido'] 

# pirapama_df.drop(columns=['Data'], inplace=True)

# pirapama_df.drop(columns=['kWh fornecido'], inplace=True)

# X_pirapama = pirapama_df['index'].values.reshape(-1,1)
# y_pirapama = pirapama_df['kwh'].values

# y_pred = regr.predict(X_pirapama)

# plt.figure(figsize=(8, 6))
# plt.plot(X_pirapama, y_pirapama, color="blue", label="Valores Reais")
# plt.plot(X_pirapama, y_pred, color="red", label="Valores Previstos")
# plt.title(f"Hora da Verdade")
# plt.xlabel("Hora (X)")
# plt.ylabel("Consumo (y)")
# plt.legend()
# plt.show()

In [7]:
# def identify_peaks_and_valleys(y_test):

#     # Inicializa os pesos
#     weights = tf.ones_like(y_test)

#     # Obtém a forma do tensor
#     length = tf.shape(y_test)[0]  # comprimento do vetor

#     # Itera sobre o vetor para encontrar picos e vales
#     for i in range(1, length - 1):
#         if y_test[i] > y_test[i - 1] and y_test[i] > y_test[i + 1]:
#             # Se é um pico
#             weights = tf.tensor_scatter_nd_update(weights, [[i]], [3.0])  # Aumenta o peso
#         elif y_test[i] < y_test[i - 1] and y_test[i] < y_test[i + 1]:
#             # Se é um vale
#             weights = tf.tensor_scatter_nd_update(weights, [[i]], [3.0])  # Aumenta o peso

#     # Verificar o primeiro elemento
#     if length >= 3:  # Verifica se existem elementos suficientes
#         if (y_test[0] > y_test[1]) and (y_test[0] > y_test[2]):  # Se o primeiro valor for maior que o segundo e o terceiro, é um pico
#             weights = tf.tensor_scatter_nd_update(weights, [[0]], [3.0])  # Aumenta o peso
#         elif (y_test[0] < y_test[1]) and (y_test[0] < y_test[2]):  # Se for menor, é um vale
#             weights = tf.tensor_scatter_nd_update(weights, [[0]], [3.0])  # Aumenta o peso

#     # Verificar o último elemento
#     if length >= 3:  # Verifica se existem elementos suficientes
#         if (y_test[-1] > y_test[-2]) and (y_test[-1] > y_test[-3]):  # Se o último valor for maior que o penúltimo e o antepenúltimo, é um pico
#             weights = tf.tensor_scatter_nd_update(weights, [[length - 1]], [3.0])  # Aumenta o peso
#         elif (y_test[-1] < y_test[-2]):  # Se for menor, é um vale
#             weights = tf.tensor_scatter_nd_update(weights, [[length - 1]], [3.0])  # Aumenta o peso

#     return weights

In [8]:
### FUNÇÃO PARA IDENTIFICAR PICOS E VALES SEM AVALIAÇÃO DO PRIMEIRO E ÚLTIMO ELEMENTOS ###

# import tensorflow as tf

# def identify_peaks_and_valleys(y_test):
#     # Converte y_test em um tensor
#     y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
#     # Inicializa pesos como um tensor de 1s do mesmo formato
#     weights = tf.ones(tf.shape(y_test), dtype=tf.float32)

#     # Obtenha o número de elementos
#     num_elements = tf.shape(y_test)[0]

#     # Obter os índices de picos e vales
#     y_test_prev = tf.roll(y_test, shift=1, axis=0)
#     y_test_next = tf.roll(y_test, shift=-1, axis=0)

#     # Define picos e vales
#     peaks = tf.where((y_test > y_test_prev) & (y_test > y_test_next))
#     valleys = tf.where((y_test < y_test_prev) & (y_test < y_test_next))

#     # Atualiza os pesos para picos e vales
#     weights = tf.tensor_scatter_nd_update(weights, peaks, tf.fill(tf.shape(peaks)[:1], 3.0))
#     weights = tf.tensor_scatter_nd_update(weights, valleys, tf.fill(tf.shape(valleys)[:1], 3.0))

#     return weights


In [9]:
import tensorflow as tf

def identify_peaks_and_valleys(y_test):
    # Converte y_test em um tensor
    #y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
    # Inicializa pesos como um tensor de 1s do mesmo formato
    weights = tf.ones(tf.shape(y_test), dtype=tf.float32)

    # Condicional para verificar se y_test tem pelo menos 3 elementos
    mask = tf.size(y_test) < 3
    #print(tf.size(y_test) < 3)

    # Função para processar picos e vales
    def process_peaks_and_valleys():
        # Certifique-se de que y_test é do tipo float32
        y_test_float = tf.cast(y_test, tf.float32)

        # Obtém os índices de picos e vales para elementos intermediários
        # Corrigido: use tf.expand_dims para garantir que o primeiro elemento tenha o formato correto
        y_test_prev = tf.concat([tf.expand_dims(y_test_float[0], axis=0), y_test_float[:-1]], axis=0)  # Adiciona o primeiro elemento como anterior
        y_test_next = tf.concat([y_test_float[1:], tf.expand_dims(y_test_float[-1], axis=0)], axis=0)  # Adiciona o último elemento como próximo

        # Define picos e vales para os elementos intermediários
        peaks = tf.where((y_test_float > y_test_prev) & (y_test_float > y_test_next))
        valleys = tf.where((y_test_float < y_test_prev) & (y_test_float < y_test_next))
        # print(f"picos",peaks)
        # print(f"vales",valleys)
        # Atualiza os pesos para picos e vales intermediários
        weights_updated = tf.tensor_scatter_nd_update(weights, peaks, tf.fill(tf.shape(peaks)[:1], 2.0))
        weights_updated = tf.tensor_scatter_nd_update(weights_updated, valleys, tf.fill(tf.shape(valleys)[:1], 2.0))

    #     # Verificação para o primeiro elemento
    #     first_idx = 0
    #     first_elem = tf.gather(y_test, first_idx)
    #     second_elem = tf.gather(y_test, first_idx + 1)
    #     third_elem = tf.gather(y_test, first_idx + 2)

    #     # Atualiza o peso do primeiro elemento
    #     first_weight_update = tf.where(
    #         (first_elem < second_elem) & (first_elem < third_elem) |
    #         (first_elem > second_elem) & (first_elem > third_elem),
    #         3.0,
    #         weights_updated[first_idx]
    #     )
    #     weights_updated = tf.tensor_scatter_nd_update(weights_updated, [[first_idx]], [first_weight_update])


    #     # Verificação para o último elemento
    #     last_idx = tf.shape(y_test)[0] - 1
    #     last_elem = tf.gather(y_test, last_idx)
    #     second_last_elem = tf.gather(y_test, last_idx - 1)
    #     third_last_elem = tf.gather(y_test, last_idx - 2)

    # # Atualiza o peso do último elemento
    #     last_weight_update = tf.where(
    #         (last_elem < second_last_elem) & (last_elem < third_last_elem) |
    #         (last_elem > second_last_elem) & (last_elem > third_last_elem),
    #         3.0,
    #         weights_updated[last_idx]
    #     )
    #     weights_updated = tf.tensor_scatter_nd_update(weights_updated, [[last_idx]], [last_weight_update])

        return weights_updated

    # Condicional para executar a função de processamento se y_test tiver 3 ou mais elementos
    weights = tf.cond(mask, lambda: weights, process_peaks_and_valleys)

    return weights


In [10]:
# Função WMSE ajustada com pesos para picos e vales
def wmse(y_test, y_pred):
    # Convertendo y_test e y_pred para float32
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    # Removendo dimensões desnecessárias
    #y_test = tf.squeeze(y_test)
    #y_pred = tf.squeeze(y_pred)

    # Obtendo pesos
    weights = identify_peaks_and_valleys(y_test)
    #tf.print("Pesos (weights) no WMSE:", weights)

    # Calculando WMSE
    wmse_value = (tf.reduce_sum(weights * tf.square(y_test - y_pred)) / tf.reduce_sum(weights))
    return wmse_value

In [11]:
# Função WMAE ajustada com pesos para picos e vales
def wmae(y_test, y_pred):
    # Convertendo y_test e y_pred para float32
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    #y_test = tf.squeeze(y_test)
    #y_pred = tf.squeeze(y_pred)

    weights = identify_peaks_and_valleys(y_test)
    #tf.print("Pesos (weights) no WMAE:", weights)

    # Calculando WMAE
    wmae_value = (tf.reduce_sum(weights * tf.abs(y_test - y_pred)) / tf.reduce_sum(weights))
    return wmae_value

In [12]:
# print(y_test[5])
# weights = identify_peaks_and_valleys(y_test[5])
# print(weights)


In [13]:


scaler = MinMaxScaler(feature_range=(0, 1))  # Padronização dos dados
scaled_kwh = scaler.fit_transform(duas_unas_df['kwh'].values.reshape(-1, 1))  # padronização de duas_unas_df

scaled_kwh_validation = scaler.fit_transform(duas_unas_validation['kwh'].values.reshape(-1,1))  # padronização de duas_unas_validacao

def create_dataset(data, time_step, output_steps):  # Função para construção do dataset
    X, y = [], []  # Criação das listas
    for i in range(len(data) - time_step - output_steps + 1):  # Loop
        X.append(data[i:(i + time_step), 0])  # Construção do dataset 
        y.append(data[(i + time_step):(i + time_step + output_steps), 0])
    return np.array(X), np.array(y)

# Create the dataset
X, y = create_dataset(scaled_kwh, time_step, output_steps)

X_validation, y_validation = create_dataset(scaled_kwh_validation, time_step, output_steps)

# Reshape input to be [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)
X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], 1)

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# print(y_test)
# print(X_test.shape)

# # Build the LSTM model
# model = Sequential()
# model.add(LSTM(20, return_sequences=True, input_shape=(X_train.shape[1], 1)))
# model.add(Dropout(0.2))
# #model.add(LSTM(10, return_sequences=True))
# #model.add(Dropout(0.2))
# model.add(LSTM(20, return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(output_steps))

# learning_rate = 0.0005
# optimizer = Adam(learning_rate=learning_rate)

# model.compile(optimizer=optimizer, loss='mean_squared_error')

# #callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.02, patience=2, verbose=0, mode='auto')
# history = model.fit(X_train, y_train, epochs=100000, batch_size=4, validation_split=.2, callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=2, patience=3))

# # Train the model
# #history = model.fit(X_train, y_train, epochs=1000, batch_size=4, verbose=1, callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1))

# # Plotar as perdas de treinamento e validação
# plt.plot(history.history['loss'], label='Loss de Treinamento')
# plt.plot(history.history['val_loss'], label='Loss de Validação')
# plt.xlabel('Épocas')
# plt.ylabel('Perda')
# plt.legend()
# plt.show()

In [14]:
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.callbacks import EarlyStopping
from keras.activations import relu, tanh, sigmoid

# Treina o modelo LSTM com os hiperparâmetros fornecidos pela partícula do PSO e avalia o modelo
def fitness_function(particle):
    units = int(particle[0])
    dropout_rate = particle[1]
    learning_rate = particle[2]
    batch_size = int(particle[3])
    #epochs = int(particle[4])
    patience = int(particle[4])
    num_camadas_LSTM = int(particle[5])
    activation_idx = 1 #int(particle[6])
    loss_idx = 1 #int(particle[7])
    optimizer_idx = 1 #int(particle[8])
    
    print(f"units", units)
    print(f"dropout_rate", dropout_rate)
    print(f"learning_rate", learning_rate)
    print(f"batch_size", batch_size)
    print(f"patience", patience)
    print(f"num_camadas_LSTM", num_camadas_LSTM)

    # Mapeamentos para as métricas de perda, otimizadores e ativação
    loss_options = [wmse, wmae, "mean_squared_error", "mean_absolute_error"]
    optimizer_options = ["adam", "rmsprop", "sgd"]
    activation_options = ["tanh", "relu", "sigmoid"]

    # Mapear índices para os parâmetros
    loss = loss_options[loss_idx]  # índice da função de perda
    optimizer = optimizer_options[optimizer_idx]  # índice do otimizador
    activation = activation_options[activation_idx]  # índice da função de ativação

    print(f"activation", activation)
    print(f"loss", loss)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Criar o modelo
    model = Sequential()

    # Laço para adicionar o número de camadas LSTM e dropout especificado por num_camadas_LSTM
    for _ in range(int(num_camadas_LSTM)):
        model.add(LSTM(int(units), return_sequences=True if _ < int(num_camadas_LSTM) - 1 else False, input_shape=(X_train.shape[1], 1)))
        # A relação condicional define se a camada deve retornar a sequência completa ou somente 
        # o último estado. Apenas a última camada LSTM não retorna sequência, porque é a camada final
        model.add(Dropout(dropout_rate))

    model.add(Dense(output_steps, activation=activation))
    # camada de saída em que "out_steps" define o número de passos futuros que serão previstos

    # Selecionar o otimizador
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:  # 'sgd'
        opt = SGD(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss=loss)  # Compilação do modelo definição da função de perda

    # Treinamento do modelo salvamento de 'loss' e 'val_loss' para posterior plot
    early_stopping = EarlyStopping(monitor='val_loss', patience=int(patience), restore_best_weights=True)
    # EarlyStopping: Importa o callback EarlyStopping, que para o treinamento se a perda de validação
    # (val_loss) não melhorar após "patience" épocas. O parâmetro "restore_best_weights=True" garante
    # que os pesos do melhor modelo serão restaurados.
    
    
    
    # k_folds = 2
    # shuffle = False
    # kf = KFold(n_splits=min(k_folds, len(X_train)), shuffle=shuffle)
    # for train_index, val_index in kf.split(X_train):
    #     X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    #     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    #     # X_train, X_val = X_train[train_index], X_train[val_index]
    #     # y_train, y_val = y_train[train_index], y_train[val_index]
    #     history = model.fit(X_train_fold, y_train_fold, epochs=int(100000), batch_size=int(batch_size), validation_data=(X_val_fold, y_val_fold), shuffle=shuffle, verbose=0, callbacks=[early_stopping])


    history = model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(100000), validation_data=(X_validation, y_validation), 
                        verbose=0, callbacks=[early_stopping])
    # Modelo é treinado até que a condição do EarlyStopping seja alcançada. O "loss" e "val_loss" são salvos
    #em "history" para plot posterior.

    # Plotar as perdas de treinamento e validação
    plt.plot(history.history['loss'], label='Loss de Treinamento')
    plt.plot(history.history['val_loss'], label='Loss de Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.show()
    
    # # Avaliar o modelo
    # score = model.evaluate(X_validation, y_validation, verbose=0)
    # print(f"score: {score}")

    # # Avaliação do desempenho no conjunto de validação
    # #y_pred = model.predict(X_validation) # Previsão
    
    return model.evaluate(X_validation, y_validation) # Cálculo do erro


In [ ]:
import numpy as np

# Definições para PSO
n_particles = 15   # Número de partículas (conjunto-candidato de hiperparâmetros) otimizadas por iteração
n_iterations = 30  # Número de vezes que o PSO otimiza as partículas
w = 0.5            # Inércia - controla a influência da velocidade anterior da partícula
c1 = 0.5           # Coeficiente cognitivo - controla a influência da melhor posição da partícula
c2 = 0.5           # Coeficiente social - controla a influência da melhor posição global

# Limites dos hiperparâmetros (neurônios, dropout, learning rate, batch size, patience,num_camadas_LSTM, activation, loss)
bounds = [(10, 200), (0.1, 0.4), (0.00001, 0.005), (16, 64), (2, 8), (1, 5)] #, (0, 2), (0, 1)]
bounds = [(0, 3)]

# Inicialização das partículas
particles = np.random.rand(n_particles, len(bounds))                                                     
# Gera uma matriz de números aleatórios no intervalo [0,1). A matriz 
# tem dimensões n_particles - linhas e len(bounds) - colunas, sendo len(bounds)
# o número de hiperparâmetros sendo otimizados.
particles = particles * (np.array([ub - lb for lb, ub in bounds])) + np.array([lb for lb, ub in bounds]) 
# Cada linha de particles representa uma partícula, e cada coluna representa um valor correspondente a um hiperparâmetro.
# O primeiro np.array cria um array que contém a diferença entre o limite superior (ub)
# e o limite inferior (lb) de cada hiperparâmetro. Após isso, particles (definidas inicialmente entre [0,1))
# são multiplicadas pela amplitude de cada intervalo.
# O segundo np.array cria um array com os valores mínimos de cada hiperparâmetro. Esse array é somado ao primeiro array
# para garantir que todas as partículas terão seus hiperparâmetros dentro dos limites estabelecidos. 


velocities = np.random.rand(n_particles, len(bounds)) # Define as velocidades iniciais das partículas de forma aleatória
pbest_positions = particles.copy()                    # Guarda a melhor posição individual de cada partícula
pbest_scores = [float('inf')] * n_particles  # Inicialização com um valor alto
#pbest_scores = np.full(n_particles, np.inf)           # Inicializa os melhores scores individuais como infinito
gbest_position = None                                 # A melhor posição global encontrada por todas as partículas (inicialmente None)
gbest_score = np.inf                                  # O melhor score global, inicialmente infinito

# Função para atualizar a posição e a velocidade das partículas
# Ajusta as partículas para que se movam em direção à melhor solução encontrada até o momento, 
# tanto individualmente (pbest_positions) quanto globalmente (gbest_position).
def update_particles(particles, velocities):
    # particles - array com as posições atuais de todas as partículas
    # velocities - array com as velocidades atuais de todas as partículas
    for i in range(n_particles):            # Loop para atualizar todas as partículas
        velocities[i] = w * velocities[i] + c1 * np.random.rand() * (pbest_positions[i] - particles[i]) + c2 * np.random.rand() * (gbest_position - particles[i])
        # Atualiza a velocidade da partícula. W (inercia) multiplica a velocidade atual da partícula.
        # C1 (componente cognitiva) multiplica a diferença entre a melhor posição e a posição atual (da partícula).
        # np.random.rand() gera uma matriz de números aleatórios entre [0,1) para evitar que a partícula fique presa em uma trajetória fixa
        # C2 - (componente social) multiplica a diferença entre a melhor posição considerando todas as partículas
        # e a posição atual da partícula.        
        particles[i] += velocities[i]
        # A nova posição da partícula é atualizada somando a posição atual à nova velocidade
        # Move a partícula em direção à sua nova posição no espaço de soluções
        particles[i] = np.clip(particles[i], [lb for lb, ub in bounds], [ub for lb, ub in bounds])
        # np.clip garante que cada partícula permaneça dentro dos limites definidos em bounds para cada hiperparâmetro
        # np.clip(x, a, b) faz com que, se x for menor que a, ele se torne a, e se for maior que b, ele se torne b.

# Otimização
# O PSO irá atualizar as partículas e verificar as melhores soluções n_iterations vezes.
for iteration in range(n_iterations):
    # Para cada iteração todas as partículas são avaliadas: seu desempenho e ajuste de sua posição e velocidade
    for i in range(n_particles):
        score = fitness_function(particles[i])
        # Função que treina e testa o modelo com os hiperparâmetros da partícula atual
        # Cada partícula é avaliada através da sua posição/estado atual (valores-candidatos dos seus hiperparâmetros)
        # É retornado o erro que avalia quão boa é a posição atual

        if score < pbest_scores[i]:
            pbest_scores[i] = score
            pbest_positions[i] = particles[i].copy()
        # Se o score for o melhor score pessoal da partícula, este valor e a posição (hiperparâmetros) são salvos
        # copy() é usado para garantir que uma nova instância da posição seja armazenada, evitando modificações acidentais.

        if score < gbest_score:
            gbest_score = score
            gbest_position = particles[i].copy()
            print(f"best_score_until_now", gbest_score)
            print("Melhor solução encontrada até agora:", gbest_position)
        # Se o score for o melhor score global de todas as partículas, este valor e a posição (hiperparâmetros) são salvos

    update_particles(particles, velocities)
    # Após a avaliação de todas as partículas e seus hiperparâmetros (dentro da mesma iteração), a função update_particles
    # é chamada para atualizar as posições das partículas (hiperparâmetros) com base nos resultados da iteração atual 

print("Melhor solução encontrada:", gbest_position)
print("Melhor score:", gbest_score)


In [ ]:
print("Melhor solução encontrada:", gbest_position)
print("Melhor score:", gbest_score)

In [17]:
def avaliacao_multistep(janela_real, janela_prevista):
    
    mae_med = 0
    mse_med = 0
    rmse_med = 0
    mape_med = 0
    acuracia_med = 0
    # Converte as listas para arrays NumPy
    #janela_real = np.array(janela_real)
    #janela_prevista = np.array(janela_prevista)
    print(len(janela_real[0]))
    
    for j in range(len(janela_prevista)):
        if len(janela_real) != len(janela_prevista):
            raise ValueError("As janelas reais e previstas devem ter o mesmo comprimento.")

        else:
        #r2 =  round(r2_score(janela_real, janela_prevista), 2)   # Relacionado à regressão linear dos valores verdadeiros. É calculado com base no erro quadrático residual e erro total. O valor pode ser interpretado como a proporção dos valores previstos que estão na linha criada pela regressão.
        #print("R^2 Score of LSTM model = ",r2)
        #print(" ")

            mae = round(mean_absolute_error(janela_real[j], janela_prevista[j]), 3) # Arredondamento com três casas decimais
            print("MAE(Erro absoluto médio) Score of LSTM model = ",mae)
            print(" ")

            mse = round(mean_squared_error(janela_real[j], janela_prevista[j]), 3)
            print("MSE(Erro quadrático médio) Score of LSTM model = ",mse)
            print(" ")
            
            rmse = round(np.sqrt(mse), 3)
            print("RMSE(Raiz do erro quadrático médio) Score of LSTM model = ",rmse)
            print(" ")

            mape = round(mean_absolute_percentage_error(janela_real[j], janela_prevista[j]), 2)
            print("MAPE(Erro percentual médio absoluto) Score of LSTM model = ",mape)
            print(" ")
            
            ### Cálculo da ACURÁCIA ###
            
            # Calcula a diferença percentual
            diferencas_percentuais = np.abs((np.array(janela_real[j]) - np.array(janela_prevista[j])) / np.array(janela_real[j]))

            # Conta quantas diferenças percentuais estão dentro da tolerância
            acertos = np.sum(diferencas_percentuais <= 0.05) #0.05 é a tolerância

            # Calcula a acurácia
            acuracia = acertos / len(janela_real[0])   
            acuracia = round(100 * acuracia, 2)
            
            print("Accuracy Score of LSTM model = ",acuracia)
            mae_med += mae
            mse_med += mse
            rmse_med += rmse
            mape_med += mape
            acuracia_med += acuracia
            
    mae_med = mae_med/(len(janela_prevista))
    mse_med /=(len(janela_prevista))
    rmse_med /=(len(janela_prevista))
    mape_med /=(len(janela_prevista))
    acuracia_med /=(len(janela_prevista))
    
    return round(mae_med,2), round(mse_med,2), round(rmse_med,2), round(mape_med,2), round(acuracia_med,2)

In [ ]:
y_pred = model.predict(X_validation)
print(len(X_validation), len(y_pred), len(X_validation[0]))
y_pred_rescaled = scaler.inverse_transform(y_pred)

y_test_rescaled = scaler.inverse_transform(y_validation)
#indices_to_validate = indices_to_validate[:len(y_pred_rescaled)]
indices_to_validate = indices_to_validate[:validation_interval]
print(indices_to_validate)

plt.figure(figsize=(8,6))
plt.plot(indices_to_validate[0:output_steps], y_test_rescaled[0], label="Original", color='blue')
plt.plot(indices_to_validate[0:output_steps], y_pred_rescaled[0], label="Predição", color='red')
plt.yticks(np.arange(0,200000,step=10000))
plt.show()

plt.figure(figsize=(8,6))
plt.plot(indices_to_validate[1:1+output_steps], y_test_rescaled[1], label="Original", color='blue')
plt.plot(indices_to_validate[1:1+output_steps], y_pred_rescaled[1], label="Predição", color='red')
plt.yticks(np.arange(0,200000,step=10000))
plt.show()


plt.figure(figsize=(8,6))
plt.plot(indices_to_validate[2:2+output_steps], y_test_rescaled[2], label="Original", color='blue')
plt.plot(indices_to_validate[2:2+output_steps], y_pred_rescaled[2], label="Predição", color='red')
plt.yticks(np.arange(0,200000,step=10000))
plt.show()


plt.figure(figsize=(8,6))
plt.plot(indices_to_validate[3:3+output_steps], y_test_rescaled[3], label="Original", color='blue')
plt.plot(indices_to_validate[3:3+output_steps], y_pred_rescaled[3], label="Predição", color='red')
plt.yticks(np.arange(0,200000,step=10000))
plt.show()


plt.figure(figsize=(8,6))
plt.plot(indices_to_validate[4:4+output_steps], y_test_rescaled[4], label="Original", color='blue')
plt.plot(indices_to_validate[4:4+output_steps], y_pred_rescaled[4], label="Predição", color='red')
plt.yticks(np.arange(0,200000,step=10000))
plt.show()


In [ ]:
avaliacao_multistep(y_test_rescaled,y_pred_rescaled)